<a href="https://colab.research.google.com/github/ryannemilligan/Movie-Recommendation-System/blob/main/Movie_Recommendation_Assignment_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommendation Systems for Movies

Two MovieLens datasets are used here. (link : https://www.kaggle.com/rounakbanik/the-movies-dataset/data)



In [1]:
# newer version of numpy doesn't work with surprise in colab. downgrading
!pip install numpy==1.24.4 --force-reinstall

  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.9.0 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires 

In [2]:
!pip install surprise


In [3]:
#importing required libraries
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None

from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD #(if this code doesn't run, please remove the "#" before pip install surprise in the previous cell and run the cell)
from surprise.model_selection import cross_validate


import warnings; warnings.simplefilter('ignore')

## Content Based Recommendation model

![](https://johnolamendy.files.wordpress.com/2015/10/01.png)

** Limitation of Popularity model ** <br>
It gives the same recommendation to everyone, regardless of the user's personal interest. <br>

_For example:_ If a person who loves romantic movies were to look at Top 15 romantic movies, and he wouldn't probably like most of the listed movies. So, he were to go one step further and look at movie lists by genre, he wouldn't still be getting the interesting recommendations.

Therefore, let's build a model that computes similarity between movies based on certain metrics and suggests movies that are most similar to a particular movie that a user liked. For that we have to consider metadata (or content), hence, it also known as **Content Based Filtering.**

Two Content Based Recommendation is implemented based on different contents:
1. Description Based (content: Movie Overviews and Taglines)
2. Meta Data Based (content : Movie Cast, Crew, Keywords and Genre)

** Note **: A small movie data set is used due to limiting computing power available to me.

In [4]:
#Please enter the file directory name where you have stored the dataset (make sure you store the jupyter notebook in the same folders with datasets)
small_mdf = pd.read_csv('links_small.csv')
small_mdf = small_mdf[small_mdf['tmdbId'].notnull()]['tmdbId'].astype('int')

Before extracting small data set, we need to make sure that the ID column of our main dataframe is clean and of type integer. To do this, let us try to perform an integer conversion of our IDs and if an exception is raised,we will replace the ID with NaN. We will then proceed to drop these rows from our dataframe.

In [5]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [6]:
#Please enter the file directory name where you have stored the dataset (make sure you store the jupyter notebook in the same folders with datasets)
m_df = pd.read_csv('movies_metadata.csv')
m_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [7]:
m_df['id'] = m_df['id'].apply(convert_int)

In [8]:
m_df[m_df['id'].isnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",NaN,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,False,6.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,False,7.0,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",NaN,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,False,4.3,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
m_df = m_df.drop([19730, 29503, 35587])
m_df['id'] = m_df['id'].astype('int')

In [10]:
sm_df = m_df[m_df['id'].isin(small_mdf)]
sm_df.shape

(9099, 24)

**9099** movies avaiable in our small movies metadata dataset

### 1. Description Based Recommendation

In [11]:
sm_df['tagline'] = sm_df['tagline'].fillna('')
sm_df['description'] = sm_df['overview'] + sm_df['tagline']
sm_df['description'] = sm_df['description'].fillna('')

#### Compute TF-IDF matrix

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    min_df=1,
    stop_words='english'
)

tfidf_matrix = tf.fit_transform(sm_df['description'])
print(tfidf_matrix.shape)


(9099, 268124)


#### Cosine Similarity

The Cosine Similarity is used to calculate a numeric quantity that denotes the similarity between two movies. Mathematically, it is defined as follows:

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $

Since the TF-IDF Vectorizer is used, calculating the Dot Product will directly give us the Cosine Similarity Score. Therefore, sklearn's **linear_kernel** is used instead of cosine_similarities as it's much faster.

In [13]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [14]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

We now have a pairwise cosine similarity matrix for all the movies in our dataset. The next step is to write a function that returns the 50 most similar movies based on the cosine similarity score.

In [15]:
sm_df = sm_df.reset_index()
titles = sm_df['title']
indices = pd.Series(sm_df.index, index=sm_df['title'])

In [16]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:51]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

Let us now try and get the top 10 recommendations for a few movies.

In [17]:
get_recommendations('The Godfather').head(10)

,title
973,The Godfather: Part II
8387,The Family
3509,Made
4196,Johnny Dangerously
29,Shanghai Triad
5667,Fury
2412,American Movie
1582,The Godfather: Part III
4221,8 Women
2159,Summer of Sam


In [18]:
get_recommendations('The Dark Knight').head(10)

,title
7931,The Dark Knight Rises
132,Batman Forever
1113,Batman Returns
8227,"Batman: The Dark Knight Returns, Part 2"
7565,Batman: Under the Red Hood
524,Batman
7901,Batman: Year One
2579,Batman: Mask of the Phantasm
2696,JFK
8165,"Batman: The Dark Knight Returns, Part 1"


We see that for **The Dark Knight**, our system is able to identify it as a Batman film and subsequently recommend other Batman films as its top recommendations. But unfortunately, that is all this system can do at the moment. This is not of much use to most people as it doesn't take into considerations very important features such as cast, crew, director and genre, which determine the rating and the popularity of a movie. Someone who liked **The Dark Knight** probably likes it more because of Nolan and would hate **Batman Forever** and every other substandard movie in the Batman Franchise.

Therefore, we are going to use much more suggestive metadata than **Overview** and **Tagline**. In the next subsection, Let's build a more sophisticated recommender that takes **genre**, **keywords**, **cast** and **crew** into consideration.

## Collaborative Filtering

![](https://cdn-images-1.medium.com/max/1600/1*6_NlX6CJYhtxzRM-t6ywkQ.png)



** Limitation of content based recommendation**:
- It is only capable of suggesting movies which are *close* to a certain movie. That is, it is not capable of capturing interest and providing recommendations across genres.

- It doesn't capture the personal intrest and biases of a user. Anyone querying on model for recommendations based on a movie will receive the same recommendations for that movie, regardless of who he is.

Therefore, in this section, we will use a technique called **Collaborative Filtering** to make recommendations to Movie Watchers. Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will like a particular product or service those users have used/experienced but I have not.

We will use the **Surprise** library that used extremely powerful algorithms like **Singular Value Decomposition (SVD)** to minimise RMSE (Root Mean Square Error) and give great recommendations.

In [19]:
reader = Reader()


In [20]:
#Please enter the file directory name where you have stored the dataset (make sure you store the jupyter notebook in the same folders with datasets)
ratings = pd.read_csv('ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [21]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


In [22]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

{'test_rmse': array([0.906309  , 0.8928957 , 0.90155577, 0.89682708, 0.88944698]),
 'test_mae': array([0.69543597, 0.68619505, 0.69362923, 0.6886844 , 0.68812948]),
 'fit_time': (3.287137985229492,
  5.014575958251953,
  2.814289093017578,
  1.7122833728790283,
  1.3850758075714111),
 'test_time': (0.2590939998626709,
  0.5757458209991455,
  0.35982489585876465,
  0.10429739952087402,
  0.2370738983154297)}

We get a mean **Root Mean Sqaure Error** of 0.8965 which is more than good enough right now. Let us now train on dataset and arrive at predictions.

Let us pick user 5000 and check the ratings he has given.

In [23]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [24]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.7521655428636222, details={'was_impossible': False})

For movie with ID 302, we get an estimated prediction of **2.74**. One startling feature of this recommender system is that it doesn't care what the movie is (or what it contains). It works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.

## Hybrid Recommender


we build a simple hybrid recommender that brings together techniques we have implemented in the content based and collaborative filter based engines. This is how it will work:

* **Input:** User ID and the Title of a Movie
* **Output:** Similar movies sorted on the basis of expected ratings by that particular user.

In [25]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [26]:
#Please enter the file directory name where you have stored the dataset (make sure you store the jupyter notebook in the same folders with datasets)
id_map = pd.read_csv('links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(sm_df[['title', 'id']], on='id').set_index('title')
#id_map = id_map.set_index('tmdbId')

In [27]:
indices_map = id_map.set_index('id')

In [28]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']

    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    movies = sm_df.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [29]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,id,est
975,A Grand Day Out,199.0,7.4,530,3.419670
2059,The Matrix,9079.0,7.9,603,3.367176
6105,A Trip to the Moon,314.0,7.9,775,3.001614
3360,The Dish,62.0,6.6,5257,2.943093
2854,The Hidden,85.0,6.7,12476,2.942469
2910,Pandora and the Flying Dutchman,19.0,6.5,38688,2.848849
1898,A Simple Plan,191.0,6.9,10223,2.809702
3018,Rocketship X-M,15.0,5.1,37744,2.738798
4804,Avalon,93.0,6.8,10881,2.699383
7050,Pride and Glory,243.0,6.3,13150,2.677870


In [30]:
hybrid(500, 'Avatar')

,title,vote_count,vote_average,id,est
975,A Grand Day Out,199.0,7.4,530,3.580936
6105,A Trip to the Moon,314.0,7.9,775,3.578625
5044,The Men,18.0,6.5,1882,3.135423
4804,Avalon,93.0,6.8,10881,3.080294
3360,The Dish,62.0,6.6,5257,3.072162
7587,The American,488.0,5.8,27579,3.017665
1437,Species II,161.0,4.5,10216,2.994679
1898,A Simple Plan,191.0,6.9,10223,2.994396
2910,Pandora and the Flying Dutchman,19.0,6.5,38688,2.959782
7460,Green Zone,730.0,6.4,22972,2.914450


We see that for our hybrid recommender, we get different recommendations for different users although the movie is the same. Hence, our recommendations are more personalized and tailored towards particular users.

## Conclusion

In this notebook, we have built 4 different recommendation engines based on different ideas and algorithms. They are as follows:

1. **Content Based Recommender:** We built two content based engines; one that took movie overview and taglines as input and the other which took metadata such as cast, crew, genre and keywords to come up with predictions. We also deviced a simple filter to give greater preference to movies with more votes and higher ratings.
2. **Collaborative Filtering:** We used the powerful Surprise Library to build a collaborative filter based on single value decomposition. The RMSE obtained was less than 1 and the engine gave estimated ratings for a given user and movie.
3. **Hybrid Engine:** We brought together ideas from content and collaborative filterting to build an engine that gave movie suggestions to a particular user based on the estimated ratings that it had internally calculated for that user.

# Assignment



**Task 1**

Build a *Content based recommendation system* using the following dataset in kaggle :
https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset



In [1]:
pip install pandas numpy scikit-learn

The dataset from the link above doesn't exist anymore.

I'm using a imdb dataset I found [here.](https://www.kaggle.com/datasets/simhyunsu/imdbextensivedataset)

https://www.kaggle.com/datasets/simhyunsu/imdbextensivedataset

In [2]:
# download and load the data set

import pandas as pd

df = pd.read_csv('IMDb movies.csv')

print(df.shape)
df.head()

/tmp/ipython-input-2-2242416957.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('IMDb movies.csv')


(85855, 22)


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,NaN,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,NaN,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [3]:
small_df = df.head(1000).copy()

In [4]:
# checking for useful columns

print(df.columns)

Index(['imdb_title_id', 'title', 'original_title', 'year', 'date_published',
       'genre', 'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics'],
      dtype='object')


In [8]:
# Replace missing values in small_df (not df)
for col in ['genre', 'director', 'title', 'actors', 'description']:
    small_df[col] = small_df[col].fillna('')

# Combine features in small_df itself
small_df['content'] = (
    small_df['genre'] + ' ' +
    small_df['director'] + ' ' +
    small_df['title'] + ' ' +
    small_df['actors'] + ' ' +
    small_df['description']
)


In [10]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(small_df['content'])


In [12]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

svd = TruncatedSVD(n_components=200, random_state=42)
tfidf_reduced = svd.fit_transform(tfidf_matrix)

cosine_sim = cosine_similarity(tfidf_reduced, tfidf_reduced)


In [16]:

title_to_idx = pd.Series(small_df.index, index=small_df['title']).drop_duplicates()

def recommend_movies(title, top_n=10,
                     cosine_sim=cosine_sim,
                     df=small_df,
                     index_map=title_to_idx):

    if title not in index_map:
        raise ValueError(f"Title '{title}' not found in the dataset.")

    idx = index_map[title]


    sim_scores = list(enumerate(cosine_sim[idx]))


    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1 : top_n + 1]


    movie_indices = [i for i, _ in sim_scores]


    cols = ['title', 'genre', 'director', 'actors', 'description']
    return df.loc[movie_indices, cols].reset_index(drop=True)


In [19]:
small_df['title'].sample(50).values


array(['The Greatest Question', 'Scarlet Days', 'Il re dei re',
       'Laughter', 'La corazzata Potemkin', 'Nomadi del canto',
       'Il re degli straccioni', 'Il principe studente',
       "La fille de l'eau", 'La rosa sulle rotaie', 'Outside the Law',
       'Orlacs Hände', 'Beau Brummel', "Hell's Hinges", 'Hintertreppe',
       "I dannati dell'oceano", 'The Beloved Rogue', 'The Bells',
       "One Night at Susie's", "L'uomo della Virginia", 'Sangue e arena',
       "Headin' Home", 'Eldorado', 'Assunta Spina',
       "Gli angeli dell'inferno", 'The Man from Beyond',
       'Sherlock Holmes', 'Amor gitano', 'Il re degli scapoli',
       'The Flying Scotsman', 'Furcht', 'The Great K & A Train Robbery',
       'The Pay-Off', 'Il cavaliere della libertà',
       'La ragazza del bacio', 'Paths to Paradise', 'The Love Light',
       'Il fidanzato di cartone',
       'Prividenie, kotoroe ne vozvrashchaetsya',
       'The Benson Murder Case', "La passione di Giovanna d'Arco", 'She',
      

In [21]:
recommendations = recommend_movies('Alibi', top_n=5)
recommendations

,title,genre,director,actors,description
0,The Bat Whispers,"Crime, Thriller, Horror",Roland West,"Chance Ward, Richard Tucker, Wilson Benge, DeW...",A master criminal terrorizes the occupants of ...
1,The Racketeer,"Crime, Drama, Thriller",Howard Higgin,"Robert Armstrong, Carole Lombard, Roland Drew,...",A dapper gangster sponsors an alcoholic violin...
2,Carcere,"Crime, Drama, Thriller","George W. Hill, Ward Wing","Chester Morris, Wallace Beery, Lewis Stone, Ro...",A convict falls in love with his new cellmate'...
3,The Doorway to Hell,"Crime, Drama",Archie Mayo,"Lew Ayres, Dorothy Mathews, Leon Janney, Rober...","Despite his efforts to go straight, a young ga..."
4,Dall'ombra alla luce,Drama,William Beaudine,"Loretta Young, Jack Mulhall, Raymond Hatton, G...",A young girl falls in with a gang of criminals...


**Task 2**

Build a *Collaborative recommendation system* using the following dataset in kaggle :
https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset